In [1]:
import os
import time
import pandas as pd

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

In [2]:
netunicorn_login = 'cs190n12'
netunicorn_password = 'drheuiW4'
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-cs190n12-, aws-fargate-B-cs190n12-, aws-fargate-ARM64-cs190n12-]>]


In [3]:
working_node = 'aws-fargate-A-cs190n12-'

In [4]:
pipeline = Pipeline()

# Generate data for Vimeo
pipeline.then(StartCapture(filepath="/tmp/vimeo_capture1.pcap", name="capture1"))
for _ in range(5):
    pipeline.then(WatchVimeoVideo("https://vimeo.com/375468729", 10))
pipeline.then(StopNamedCapture(start_capture_task_name="capture1"))

pipeline.then(SleepTask(2))

Pipeline(a07f5762-400d-44f9-ba65-b44886125150): {'root': [<netunicorn.library.tasks.capture.tcpdump.StartCapture object at 0x152287cd0>], 1: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152296b90>], 2: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152297b10>], 3: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152297d90>], 4: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x1522ac050>], 5: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x1522ac2d0>], 6: [<netunicorn.library.tasks.capture.tcpdump.StopNamedCapture object at 0x1522ac5d0>], 7: [<netunicorn.library.tasks.basic.SleepTask with name 6b5d7f13-b740-45da-963c-d3459e5ddd83>]}

In [5]:
pipeline.then(UploadToWebDav(filepaths={"/tmp/vimeo_capture1.pcap"}, endpoint="http://snl-server-5.cs.ucsb.edu/cs190n-test/vimeo_capture", username="test", password="test"))

Pipeline(a07f5762-400d-44f9-ba65-b44886125150): {'root': [<netunicorn.library.tasks.capture.tcpdump.StartCapture object at 0x152287cd0>], 1: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152296b90>], 2: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152297b10>], 3: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x152297d90>], 4: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x1522ac050>], 5: [<netunicorn.library.tasks.video_watchers.vimeo_watcher.WatchVimeoVideo object at 0x1522ac2d0>], 6: [<netunicorn.library.tasks.capture.tcpdump.StopNamedCapture object at 0x1522ac5d0>], 7: [<netunicorn.library.tasks.basic.SleepTask with name 6b5d7f13-b740-45da-963c-d3459e5ddd83>], 8: [<netunicorn.library.tasks.upload.webdav.UploadToWebDav object at 0x11eed6150>]}

In [6]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-cs190n12-, aws-fargate-B-cs190n12-, aws-fargate-ARM64-cs190n12-]>]


In [7]:
working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)

# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
print(experiment)

 - Deployment: Node=aws-fargate-A-cs190n12-1, executor_id=, prepared=False, error=None


In [8]:
from netunicorn.base import DockerImage
for deployment in experiment:
    # you can explore the image on the DockerHub
    deployment.environment_definition = DockerImage(image='speeeday/chromium-speedtest:0.3.1')

In [9]:
experiment_label = "datacoll"

In [10]:

client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

RemoteClientException: Failed to start experiment execution. Status code: 400, content: "Experiment datacoll not found"

In [ ]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

In [ ]:
from returns.pipeline import is_successful

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    data = result.unwrap() if is_successful(result) else result.failure()
    for key, value in data.items():
        print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

In [ ]:
df_vimeo = pd.read_csv("/mnt/md0/cs190n-test/vimeo_video_capture_ISCX.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/md0/cs190n-test/vimeo_video_capture_ISCX.csv'

In [ ]:
features = [
    "Label",
    "Protocol",
    "Flow Duration",
    "Flow Bytes/s",
    "Flow Packets/s",
    "Flow IAT Mean",
    "Bwd IAT Mean",
    "Down/Up Ratio",
    "Active Mean",
    "Idle Mean"
]

In [ ]:
df_vimeo['Label'] = 'other'
df_vimeo.loc[(df_vimeo['Total Fwd Packet'] > 30) | (df_vimeo['Total Bwd packets'] > 30), 'Label'] = 'vimeo'
df_vimeo = df_vimeo.drop(df_vimeo[(df_vimeo['Protocol'] == 17) & (df_vimeo['Label'] != 'vimeo')].index)

In [ ]:
%pip install scikit-learn

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import plot_tree

In [ ]:
target_variable = 'Label'
train_features = list(set(df.columns) - {target_variable})
x_train = df_vimeo[train_features]
y_train = df_vimeo[target_variable]

In [ ]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)


In [ ]:
y_pred = clf.predict(x_train.values)
print(metrics.classification_report(y_train, y_pred))